In [ ]:
import subprocess
import concurrent.futures

# txt文件的路径，替换为你的实际路径
file_path = 'ts.txt'
# 结果txt文件的路径
results_txt_path = 'ping_results.txt'

def ping_ipv6(ipv6_address):
    try:
        # 对每个IPv6地址执行ping命令
        output = subprocess.check_output(["ping", "-c", "1", ipv6_address], 
                                         stderr=subprocess.STDOUT, 
                                         timeout=5).decode('utf-8')  # timeout set to 500ms
        # 从输出中提取延迟时间
        latency = output.split('time=')[-1].split()[0]
        return ipv6_address
    except subprocess.CalledProcessError as e:
        print(f'Ping to {ipv6_address} failed with return code {e.returncode}')
    except subprocess.TimeoutExpired as e:
        print(f'Ping to {ipv6_address} timed out')

try:
    with open(file_path, 'r') as file, open(results_txt_path, 'w') as txtfile:
        ipv6_addresses = [line.strip() for line in file]  # assuming the IP address is in each line

        # 使用ThreadPoolExecutor并行执行ping命令
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = executor.map(ping_ipv6, ipv6_addresses)
            for result in results:
                if result is not None:
                    txtfile.write(result + '\n')
                    print(f'Successful Ping to {result}')
except FileNotFoundError:
    print("The file was not found. Please check the file path.")
except Exception as e:
    print(f'An error occurred: {e}')

In [ ]:
import requests
import time
from concurrent.futures import ThreadPoolExecutor

# 设置全局变量
TIMEOUT = 10  # 请求超时时间
URL = "https://cf.xiu2.xyz/url"  # 下载测速地址，端口号为8080
BUFFER_SIZE = 1024  # 缓冲区大小

def download_speed_test(ip):
    start_time = time.time()
    try:
        # 发送GET请求
        response = requests.get(URL, timeout=TIMEOUT, stream=True)
        total_data = 0
        for data in response.iter_content(chunk_size=BUFFER_SIZE):
            total_data += len(data)
            if time.time() - start_time > TIMEOUT:
                break
        speed = total_data / (time.time() - start_time)
        return ip, speed / (1024 * 1024)  # convert speed to MB/s
    except requests.exceptions.RequestException:
        return ip, 0

def main():
    # 读取ip.txt中的所有IP
    with open("ping_results.txt", "r") as file:
        ip_list = [line.strip() for line in file.readlines()]

    # 使用线程池进行并发测速
    with ThreadPoolExecutor(max_workers=32) as executor:
        results = list(executor.map(download_speed_test, ip_list))

    # 按速度从大到小排序结果
    sorted_results = sorted(results, key=lambda x: x[1], reverse=True)

    # 输出测速结果并保存到txt文件
    with open("speed_results.txt", "w") as file:
        file.write("ip\tspeed(MB/s)\n")  # write column names
        for result in sorted_results:
            file.write(f"{result[0]}\t{result[1]}\n")

    # 将IP地址保存到speed_ip.txt文件中
    with open("speed_ip.txt", "w") as file:
        for ip in ip_list:
            file.write(f"{ip}\n")

if __name__ == "__main__":
    main()